# Entrenamiento y evaluación del modelo: Segunda parte

Para la segunda parte, se hará uso de una red neuronal convolucional. La idea central es ver si la accuracy del modelo puede mejorar un poco. Para esto, se tendrán que realizar ciertos ajustes al modelo.

En mi modelo anterior, se alcanzaron los siguientes valores de Accuracy:

- Training Data Accuracy: 92%
- Testing Data Accuracy: 87%

### Leo la data preprocesada

In [1]:
%store -r x_train
%store -r y_train
%store -r x_test
%store -r y_test
%store -r yy
%store -r label_encoder

### Feature Extraction

Al estar trabajando con una CNN, voy a necesitar que mis inputs sean todos del mismo tamaño. Previamente, los vectores que obtenía luego del análisis de los MFCC variaban en sus dimensiones en función de la duración de los ejemplos.

Para dejar todos mis inputs con las mismas longitudes, utilizo zero-padding.

In [2]:
import numpy as np
import pandas as pd
import os
import librosa


max_pad_len = 174

# Vuelvo a definir mi función para extraer features, pero utilizando mi zero padding a 174 como máximo.

def extract_features (file_name):
    # En primer lugar, voy a testear un resampleo kaiser_fast para que sea mas rapido
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
    
    # Agrego ceros hasta la longitud deseada
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    
    return mfccs

Nuevamente, leo mi dataset y extraigo los features ahora con zero padding,

In [4]:
# Defino el path absoluto donde estan localizados todos los audios

dataset_path = '/media/diego/4A64372E64371BDF/Downloads/Guitar Pro 5.2/UrbanSound8K/audio/'

# Leo la metadata 

metadata = pd.read_csv('../Jupyter Notebooks/Metadata/UrbanSound8K.csv')

# Preparo donde voy a colocar los features

features = []

# Itero a través de cada audio para obtener los nombres de los archivos y las clases

for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    class_label = row ["class_name"]
    
    # Extraigo los features para cada archivo
    
    data = extract_features (file_name)
    
    #Appendo en features []
    
    features.append([data, class_label])
    
# Convierto a un pandas dataframe

features_df = pd.DataFrame(features, columns=['features','class_label'])

Vuelvo a convertir mis datos de numerical a categorical

In [5]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convierto los features y sus correspondientes class_labels a numpy arrays

X = np.array (features_df.features.tolist())
y = np.array (features_df.class_label.tolist())

# Transformo las classification labels a numérico

label_encoder = LabelEncoder()

# Convierto el vector y (donde tengo las class_labels) a una matriz de clases binarias (0s y 1s).
# Con label_encoder las transformo a enteros, y con to_categorical paso esos enteros a la matriz binaria.

yy = to_categorical(label_encoder.fit_transform(y))

/home/diego/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Vuelvo a splitear mi dataset

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size = 0.2, random_state = 42) 

# Convolutional Neural Network (CNN)

En esta etapa, voy a migrar de un modelo basado en un MLP a una Red Neuronal Convolucional (CNN).

Voy a utilizar nuevamente un modelo de tipo _sequential_. Inicialmente, este modelo contará con 4 capas de convolución (Conv2D). La última capa (output layer) será _densa_. 

Las capas de convolución tienen como objetivo la _detección de features_. La forma en la que trabaja es a partir de la operación básica de __convolución__ aplicada a una señal de 2 dimensiones (en este caso, las imágenes); es un proceso sencillo en el cual se desliza una ventana o filtro (antes de esto se invierte la misma) a través de todo el input, para luego realizar una multiplicación matricial y guardar los outputs en un __feature map__. Generalmente, en audio esta operación se realiza por sobre la señal temporal (considerada 1D).

El parámetro _filter_ especifica el número de nodos en cada capa. Cada capa va a incrementarse en tamaño desde 16, 32, 64 a 128 mientras el parámetro _kernel size_ determina el tamaño de la ventana o filtro (denominado __kernel window__). En este caso, este parámetro es 2, resultando en una ventana o filtro de dimensión 2x2.

La primera capa va a recibir mis inputs, que tienen una forma de (40, 174, 1) siendo 40 el número de MFCCs calculado, 174 el número de frames total teniendo en cuenta el zero padding realizado y 1 indicando que el audio es mono.

La capa de salida va a tener 10 nodos (correspondientes a los 10 labels y a las 10 posibles clasificaciones). La función de activación para la capa de salida es la misma que en el ejemplo anterior (softmax).

### Función de activación

Como función de activación para mis primeras 2 capas se utilizará __ReLU__ o Rectified Linear Activation, al igual que en el caso anterior. En lugar de utilizar un dropout del 50%, esta vez se utilizará un dropout menor (20%).

### Pooling

Las __pooling layers__ se utilizan para reducir la dimensionalidad de mi modelo (a partir de reducir ciertos parámetros). Las mismas ayudan a disminuir el tiempo de entrenamiento y también el posible overfitting que pueda existir.

Uno de los tipos de pooling layers que más se utiliza es el __Max Pooling__, que toma el tamaño máximo para cada ventana o patch de mi feature map, es decir, devuelve el valor máximo de la porción de imagen cubierta por el kernel. Otro tipo es __Average Pooling__, que devuelve justamente el promedio de la porción de imagen cubierta por el kernel.

Cada capa convolucional tiene asociada una _pooling layer_ del tipo _MaxPooling2D_, mientras que la capa convolucional final va a tener asociada una pooling layer del tipo _GlobalAveragePooling2D_. 

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

# Defino la forma de mis inputs

num_rows = 40
num_columns = 174
num_channels = 1

# Reshapeo utilizando los parámetros previamente definidos de zero padding

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)


num_labels = yy.shape[1]
filter_size = 2

# Construyo mi modelo secuencial capa por capa

model = Sequential()

# Construyo mi primer capa de inputs
model.add (Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add (MaxPooling2D(pool_size=2))
model.add (Dropout(0.2))

# Construyo mi siguiente capa
model.add (Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add (MaxPooling2D(pool_size=2))
model.add (Dropout (0.2))

# Construyo mi siguiente capa
model.add (Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add (MaxPooling2D(pool_size=2))
model.add (Dropout (0.2))

# Construyo mi siguiente capa
model.add (Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add (MaxPooling2D(pool_size=2))
model.add (Dropout(0.2))
model.add (GlobalAveragePooling2D())

# Construyo mi capa de outputs
model.add(Dense(num_labels, activation='softmax')) 

### Compilación del modelo

Para compilar el modelo, voy a utilizar los mismos 3 parámetros que utilicé en el ejemplo anterior (Categorical Cross-Entropy, Accuracy como métrica y Adam como optimizer).

In [8]:
# Compilo el modelo con los parámetros mencionados

model.compile (loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [9]:
# Muestro un resumen de lo que sería la arquitectura que estoy usando

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 86, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 41, 64)         8

In [10]:
# Calculo mi Accuracy antes del entrenamiento (pre-training accuracy)

score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Pre-training accuracy: 9.6737%


## Entrenamiento del modelo

En esta sección, se iniciará con el entrenamiento de mi modelo. Debido a que el entrenamiento de una red neuronal puede tomar bastante tiempo, se utilizarán inicialmente 10 epochs, que se refieren justamente a la cantidad de veces que mi modelo va a iterar por mi data. De esta manera, el modelo intentará mejorar en cada iteración, hasta alcanzar un cierto objetivo. Además, inicialmente se utilizará un batch size bajo también. Si se observa que el modelo comienza a converger, se incrementarán ambos parámetros.

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

number_epochs = 72

# Voy a dividir mi dataset en batchs de 128 samples
#batchsize = 128
batchsize = 256


# Con checkpoint puedo guardar el modelo o sus pesos en un determinado punto o intervalo.
checkpointer = ModelCheckpoint (filepath = '/home/diego/ML/Urban Sounds/Jupyter Notebooks/models/weights.best.basic_cnn.hdf5', verbose = 1,
                               save_best_only = True)

# Empiezo a contabilizar el tiempo para ver cuánto va a tardar en entrenar
start = datetime.now()

# Entreno mi modelo con los parámetros elegidos por un determinado número de epochs o iteraciones en mi dataset. 

model.fit(x_train, y_train, batch_size = batchsize, epochs = number_epochs, validation_data = (x_test, y_test), 
          callbacks = [checkpointer], verbose = 1)

# Veo cuánto tardó en entrenar
duration = datetime.now() - start

print ("Entrenamiento completado en: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/72
6912/6985 [============================>.] - ETA: 0s - loss: 3.5766 - accuracy: 0.1670
Epoch 00001: val_loss improved from inf to 2.02712, saving model to /home/diego/ML/Urban Sounds/Jupyter Notebooks/models/weights.best.basic_cnn.hdf5
6985/6985 [==============================] - 77s 11ms/sample - loss: 3.5603 - accuracy: 0.1691 - val_loss: 2.0271 - val_accuracy: 0.2885
Epoch 2/72
6912/6985 [============================>.] - ETA: 0s - loss: 1.9129 - accuracy: 0.3105 - ETA: 10s - loss: 1.9571 -
Epoch 00002: val_loss improved from 2.02712 to 1.82297, saving model to /home/diego/ML/Urban Sounds/Jupyter Notebooks/models/weights.best.basic_cnn.hdf5
6985/6985 [==============================] - 34s 5ms/sample - loss: 1.9104 - accuracy: 0.3112 - val_loss: 1.8230 - val_accuracy: 0.3698
Epoch 3/72
6912/6985 [============================>.] - ETA: 0s - loss: 1.6468 - accuracy: 0.4084
Epoch 00003: val_loss improved from 1.82297 to 1.66813,

Epoch 22/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.6653 - accuracy: 0.7718
Epoch 00022: val_loss did not improve from 0.72419
6985/6985 [==============================] - 41s 6ms/sample - loss: 0.6660 - accuracy: 0.7714 - val_loss: 0.7370 - val_accuracy: 0.7728
Epoch 23/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.6361 - accuracy: 0.7849
Epoch 00023: val_loss did not improve from 0.72419
6985/6985 [==============================] - 40s 6ms/sample - loss: 0.6342 - accuracy: 0.7854 - val_loss: 0.7245 - val_accuracy: 0.7705
Epoch 24/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.6273 - accuracy: 0.7840
Epoch 00024: val_loss improved from 0.72419 to 0.67168, saving model to /home/diego/ML/Urban Sounds/Jupyter Notebooks/models/weights.best.basic_cnn.hdf5
6985/6985 [==============================] - 37s 5ms/sample - loss: 0.6268 - accuracy: 0.7840 - val_loss: 0.6717 - val_accuracy: 0.7962
Epoch 25/72
6912/6985 [=================

6912/6985 [============================>.] - ETA: 0s - loss: 0.3857 - accuracy: 0.8704
Epoch 00046: val_loss did not improve from 0.43629
6985/6985 [==============================] - 40s 6ms/sample - loss: 0.3880 - accuracy: 0.8694 - val_loss: 0.4559 - val_accuracy: 0.8558
Epoch 47/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.3964 - accuracy: 0.8688
Epoch 00047: val_loss improved from 0.43629 to 0.40114, saving model to /home/diego/ML/Urban Sounds/Jupyter Notebooks/models/weights.best.basic_cnn.hdf5
6985/6985 [==============================] - 44s 6ms/sample - loss: 0.3953 - accuracy: 0.8689 - val_loss: 0.4011 - val_accuracy: 0.8724
Epoch 48/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.3581 - accuracy: 0.8746
Epoch 00048: val_loss did not improve from 0.40114
6985/6985 [==============================] - 37s 5ms/sample - loss: 0.3594 - accuracy: 0.8743 - val_loss: 0.4600 - val_accuracy: 0.8466
Epoch 49/72
6912/6985 [============================>

6912/6985 [============================>.] - ETA: 0s - loss: 0.2566 - accuracy: 0.9109
Epoch 00071: val_loss did not improve from 0.33365
6985/6985 [==============================] - 43s 6ms/sample - loss: 0.2559 - accuracy: 0.9111 - val_loss: 0.3456 - val_accuracy: 0.8918
Epoch 72/72
6912/6985 [============================>.] - ETA: 0s - loss: 0.2489 - accuracy: 0.9119
Epoch 00072: val_loss did not improve from 0.33365
6985/6985 [==============================] - 41s 6ms/sample - loss: 0.2488 - accuracy: 0.9120 - val_loss: 0.3593 - val_accuracy: 0.8867
('Entrenamiento completado en: ', datetime.timedelta(0, 2957, 26482))


In [12]:
print (duration)

0:49:17.026482


## Testeo del modelo

Voy a chequear la Accuracy de mi modelo, tanto para mi train set como para mi test set

In [13]:
# Evalúo mi modelo en mi training dataset

score_train = model.evaluate (x_train, y_train, verbose = 0)
print ("Training Accuracy: ", score_train [1])

# Evalúo mi modelo en mi testing dataset

score_test = model.evaluate (x_test, y_test, verbose = 0)
print ("Testing Accuracy: ", score_test [1])

('Training Accuracy: ', 0.94230497)
('Testing Accuracy: ', 0.88666284)


Veo que mis valores de Accuracy mejoraron un poco (ambas incrementaron un 2%).

## Predicciones

Voy a generar predicciones de una manera similar al notebook anterior.

In [16]:
# Defino una función para hacer predicciones sobre mis audios

def print_prediction (file_name):
    
    prediction_feature = extract_features(file_name)
    
    # Agrego el reshape que estuve trabajando
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)
    
    # Genero predicciones de clases para mis inputs
    predicted_vector = model.predict_classes(prediction_feature)
    
    # Transformo mis labels a mi original encoding
    predicted_class = label_encoder.inverse_transform (predicted_vector)
    
    print("The predicted class is:", predicted_class[0], '\n')
    
    
    # Determino las predicciones de probabilidad de las clases
    predicted_proba_vector = model.predict_proba(prediction_feature)
    predicted_proba = predicted_proba_vector[0]
    
    for i in range (len(predicted_proba)):
        category = label_encoder.inverse_transform(np.array([i]))
        print (category [0], "\t\t : ", format(predicted_proba[i], '.32f'))

## Validación del modelo

Hago una chequeo inicial para verificar las predicciones utilizando un subset de archivos de audio que se trabajaron en el primer notebook. Se espera que estos sean clasificados correctamente.

#### 1) Car Horn

In [17]:
filename = '../Jupyter Notebooks/Audio Samples/24074-1-0-4.wav' 
print_prediction(filename)

('The predicted class is:', 'car_horn', '\n')
('air_conditioner', '\t\t : ', '0.00000000000000000145999859657447')
('car_horn', '\t\t : ', '1.00000000000000000000000000000000')
('children_playing', '\t\t : ', '0.00000000000000000035057211999583')
('dog_bark', '\t\t : ', '0.00000000000000000030254198112552')
('drilling', '\t\t : ', '0.00000000000000068416333457387533')
('engine_idling', '\t\t : ', '0.00000000000002424382275609173604')
('gun_shot', '\t\t : ', '0.00000000000000000000000006576691')
('jackhammer', '\t\t : ', '0.00000000000000000000000872431567')
('siren', '\t\t : ', '0.00000000000000000320699722604641')
('street_music', '\t\t : ', '0.00000000013211343130592467787210')


#### 2) Dog Bark

In [18]:
filename = '../Jupyter Notebooks/Audio Samples/31323-3-0-22.wav' 
print_prediction(filename)

('The predicted class is:', 'dog_bark', '\n')
('air_conditioner', '\t\t : ', '0.00000000000002514234514214373079')
('car_horn', '\t\t : ', '0.00000000000000733597024408583556')
('children_playing', '\t\t : ', '0.00001585400605108588933944702148')
('dog_bark', '\t\t : ', '0.99998414516448974609375000000000')
('drilling', '\t\t : ', '0.00000000000000013945926314465145')
('engine_idling', '\t\t : ', '0.00000000000000227230013573180047')
('gun_shot', '\t\t : ', '0.00000000000000000000123792001338')
('jackhammer', '\t\t : ', '0.00000000000000000000058858333350')
('siren', '\t\t : ', '0.00000000011985931691604889692826')
('street_music', '\t\t : ', '0.00000000757157625486115648527630')


#### 3) Drilling

In [19]:
filename = '../Jupyter Notebooks/Audio Samples/22962-4-0-1.wav'
print_prediction(filename)

('The predicted class is:', 'car_horn', '\n')
('air_conditioner', '\t\t : ', '0.00197819457389414310455322265625')
('car_horn', '\t\t : ', '0.37578153610229492187500000000000')
('children_playing', '\t\t : ', '0.02147117629647254943847656250000')
('dog_bark', '\t\t : ', '0.03258474543690681457519531250000')
('drilling', '\t\t : ', '0.16188611090183258056640625000000')
('engine_idling', '\t\t : ', '0.06561945378780364990234375000000')
('gun_shot', '\t\t : ', '0.00102362781763076782226562500000')
('jackhammer', '\t\t : ', '0.03988300263881683349609375000000')
('siren', '\t\t : ', '0.03907153010368347167968750000000')
('street_music', '\t\t : ', '0.26070058345794677734375000000000')


Nuevamente, tengo una clasificación errónea de drilling, pero esta vez como Car Horn con 37% confidence (algo que no es completamente errado, ya que subjetivamente se percibe una bocina de auto en el audio), mientras que dirlling tiene 16%.

In [21]:
import IPython.display as ipd
ipd.Audio ('../Jupyter Notebooks/Audio Samples/22962-4-0-1.wav')

#### 4) Gun Shot

In [20]:
filename = '../Jupyter Notebooks/Audio Samples/7061-6-0-0.wav'
print_prediction (filename)

('The predicted class is:', 'gun_shot', '\n')
('air_conditioner', '\t\t : ', '0.00000000753281437226860361988656')
('car_horn', '\t\t : ', '0.00021852747886441648006439208984')
('children_playing', '\t\t : ', '0.00707544060423970222473144531250')
('dog_bark', '\t\t : ', '0.00552344368770718574523925781250')
('drilling', '\t\t : ', '0.00412965891882777214050292968750')
('engine_idling', '\t\t : ', '0.00269497954286634922027587890625')
('gun_shot', '\t\t : ', '0.97539174556732177734375000000000')
('jackhammer', '\t\t : ', '0.00000216900502891803625971078873')
('siren', '\t\t : ', '0.00495122047141194343566894531250')
('street_music', '\t\t : ', '0.00001270401662623044103384017944')


### B) Testeo con audio externo a mis datasets

In [29]:
filename = '../Jupyter Notebooks/Audio Samples/dog_bark_1.wav'
print_prediction (filename)

('The predicted class is:', 'dog_bark', '\n')
('air_conditioner', '\t\t : ', '0.00305432779714465141296386718750')
('car_horn', '\t\t : ', '0.01427096407860517501831054687500')
('children_playing', '\t\t : ', '0.02974716015160083770751953125000')
('dog_bark', '\t\t : ', '0.78981500864028930664062500000000')
('drilling', '\t\t : ', '0.11488822847604751586914062500000')
('engine_idling', '\t\t : ', '0.00201808963902294635772705078125')
('gun_shot', '\t\t : ', '0.02794492244720458984375000000000')
('jackhammer', '\t\t : ', '0.00055961654288694262504577636719')
('siren', '\t\t : ', '0.01143030729144811630249023437500')
('street_music', '\t\t : ', '0.00627130083739757537841796875000')


In [30]:
filename = '../Jupyter Notebooks/Audio Samples/gun_shot_1.wav'
print_prediction (filename)

('The predicted class is:', 'gun_shot', '\n')
('air_conditioner', '\t\t : ', '0.00001394213268213206902146339417')
('car_horn', '\t\t : ', '0.00000235012475968687795102596283')
('children_playing', '\t\t : ', '0.00039504290907643735408782958984')
('dog_bark', '\t\t : ', '0.02331313490867614746093750000000')
('drilling', '\t\t : ', '0.00773297203704714775085449218750')
('engine_idling', '\t\t : ', '0.00035392274730838835239410400391')
('gun_shot', '\t\t : ', '0.96803992986679077148437500000000')
('jackhammer', '\t\t : ', '0.00000288943488158110994845628738')
('siren', '\t\t : ', '0.00009567827510181814432144165039')
('street_music', '\t\t : ', '0.00005015299393562600016593933105')


In [32]:
filename = '../Jupyter Notebooks/Audio Samples/drilling_1.wav'
print_prediction (filename)

('The predicted class is:', 'jackhammer', '\n')
('air_conditioner', '\t\t : ', '0.00237056822516024112701416015625')
('car_horn', '\t\t : ', '0.00015262495435308665037155151367')
('children_playing', '\t\t : ', '0.00125818606466054916381835937500')
('dog_bark', '\t\t : ', '0.00067948264768347144126892089844')
('drilling', '\t\t : ', '0.00738029554486274719238281250000')
('engine_idling', '\t\t : ', '0.02585771866142749786376953125000')
('gun_shot', '\t\t : ', '0.00000154002532326558139175176620')
('jackhammer', '\t\t : ', '0.96132397651672363281250000000000')
('siren', '\t\t : ', '0.00020062497060280293226242065430')
('street_music', '\t\t : ', '0.00077503360807895660400390625000')


Aparentemente, el modelo performea de manera bastante satisfactoria, generaliza correctamente y predice bien contra data nueva (más allá de que predice drilling como jackhammer que son muy parecidos en espectro). En el próximo paso, se llevará a cabo una segunda versión de este proyecto, cambiando ciertos parámetros de preprocesamiento (modificando los valores de resampleo, cantidad de MFCCs, profundidad de bits) y ciertos parámetros de la CNN (nodos en cada capa, dropout, etc.)